In [41]:
import pandas as pd
import numpy as np
from decimal import Decimal, ROUND_HALF_UP
import tushare as ts
import time

pd.set_option('display.max_rows', 50000)  # 最多显示数据的行数
df_record = pd.DataFrame(columns=['股票代码', '股票名称', '平均涨幅', '平均跌幅', '涨次', '跌次', '平次', '共次', '涨率', '跌率', '期望'])
df_record = df_record[['股票代码', '股票名称', '平均涨幅', '平均跌幅', '涨次', '跌次', '平次', '共次', '涨率', '跌率', '期望']]

code_list = pd.read_csv('code_data/沪深A股代码和名称.csv',encoding='gbk',converters = {'代码':str})
error_list = []
for i in range(1095,code_list.shape[0]):
    
    new = np.NaN
    code = code_list.iat[i,0]
    name = code_list.iat[i,1]
    print('---------- 正在处理股票：{}, {}, {} -----------'.format(i,code,name))
    
    max_try = 10
    for _ in range(max_try):
        try:
            df = ts.get_k_data(code, ktype='D', start='2011-19-27', end='2021-09-27')
        except Exception as e:
            error_list.append(i)
            print('错误：', e, '---再次尝试中')
            time.sleep(1)
            df = pd.DataFrame(columns=['date','open','close','high','low','volume','code'])
        else:
            break
    
    df['pre_close'] = df['close'].shift(1)
    df['stop'] = df['pre_close'] * 1.1
    df['stop'] = df['stop'].apply(lambda x: float(Decimal(x*100).quantize(Decimal('1'), rounding=ROUND_HALF_UP) / 100))
    condition1 = df['high'].shift(1) > df['stop'].shift(1) - 0.005
    condition2 = df['high'].shift(1) < df['stop'].shift(1) + 0.005 # 这样 剔除复牌后的疯涨情况
    df.loc[condition1 & condition2,'signal'] = 'yesterday-stop'
    df.loc[df['signal'] == 'yesterday-stop', 'revenue'] = df['open']-df['high'].shift(1)
    df['cash_ratio'] = (df['revenue'] + df['high'].shift(1))/df['high'].shift(1)-0.003-0.0004
    df['cash_ratio']=df['cash_ratio'].fillna(value=1)
    df['cash_change'] = df['cash_ratio'].cumprod()
    df['涨跌幅'] = df['revenue']/df['high'].shift(1)
    df['涨跌幅-扣除手续费'] = df['revenue']/df['high'].shift(1)-0.003-0.0004
    df1 = df[df['涨跌幅-扣除手续费'] > 0]
    df2 = df[df['涨跌幅-扣除手续费'] < 0]
    df3 = df[df['涨跌幅-扣除手续费'] == 0]
    up_count=df1['涨跌幅-扣除手续费'].count()
    up_mean = df1['涨跌幅-扣除手续费'].mean()
    down_count=df2['涨跌幅-扣除手续费'].count()
    down_mean = df2['涨跌幅-扣除手续费'].mean()
    even_count = df3['涨跌幅-扣除手续费'].count()
    expectation = up_mean * up_count/(up_count + down_count + even_count) + down_mean * down_count/(up_count + down_count + even_count)
    
    
    new=pd.DataFrame({'股票代码': code,
                  '股票名称': name,
                  '平均涨幅': up_mean,
                  '平均跌幅': down_mean,
                  '涨次': up_count,
                  '跌次': down_count,
                  '平次': even_count,
                  '共次': up_count+down_count+even_count,
                  '涨率': up_count/(up_count+down_count+even_count),
                  '跌率': down_count/(up_count+down_count+even_count),
                  '期望': expectation
                 },index=[1]) 
    df_record=df_record.append(new)
    time.sleep(0.0333)

---------- 正在处理股票：1095, 600760, 中航沈飞 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1096, 000559, 万向钱潮 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1097, 002815, 崇达技术 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1098, 002484, 江海股份 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1099, 000532, 华金资本 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1100, 603009, 北特科技 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1101, 600405, 动力源 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1102, 600284, 浦东建设 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1103, 002039, 黔源电力 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1104, 003042, 中农联合 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/do

c:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in longlong_scalars
c:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:64: RuntimeWarning: invalid value encountered in longlong_scalars


---------- 正在处理股票：1163, 600997, 开滦股份 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1164, 003017, 大洋生物 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1165, 003006, 百亚股份 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1166, 002294, 信立泰 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1167, 603937, 丽岛新材 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1168, 002532, 天山铝业 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1169, 601528, 瑞丰银行 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1170, 603110, 东方材料 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1171, 000823, 超声电子 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1172, 600872, 中炬高新 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/do

---------- 正在处理股票：1245, 600795, 国电电力 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1246, 600778, 友好集团 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1247, 600749, 西藏旅游 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1248, 600743, 华远地产 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1249, 600723, 首商股份 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1250, 600697, 欧亚集团 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1251, 600692, 亚通股份 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1252, 600665, 天地源 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1253, 600663, 陆家嘴 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1254, 600648, 外高桥 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/docu

---------- 正在处理股票：1327, 002112, 三变科技 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1328, 002093, 国脉科技 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1329, 002050, 三花智控 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1330, 001205, 盛航股份 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1331, 000990, 诚志股份 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1332, 000955, 欣龙控股 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1333, 000952, 广济药业 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1334, 000882, 华联股份 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1335, 000875, 吉电股份 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：1336, 000862, 银星能源 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/d

...

---------- 正在处理股票：2639, 601567, 三星医疗 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2640, 000428, 华天酒店 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2641, 600787, 中储股份 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2642, 002789, 建艺集团 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2643, 603079, 圣达生物 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2644, 002157, 正邦科技 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2645, 600196, 复星医药 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2646, 002242, 九阳股份 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2647, 002088, 鲁阳节能 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2648, 000951, 中国重汽 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/d

---------- 正在处理股票：2721, 000930, 中粮科技 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2722, 603399, 吉翔股份 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2723, 600366, 宁波韵升 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2724, 601126, 四方股份 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2725, 002910, 庄园牧场 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2726, 003030, 祖名股份 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2727, 603568, 伟明环保 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2728, 600358, 国旅联合 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2729, 605117, 德业股份 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2730, 002746, 仙坛股份 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/d

---------- 正在处理股票：2803, 600283, 钱江水利 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2804, 000829, 天音控股 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2805, 603866, 桃李面包 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2806, 600965, 福成股份 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2807, 603739, 蔚蓝生物 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2808, 003043, 华亚智能 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2809, 002458, 益生股份 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2810, 002851, 麦格米特 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2811, 002634, 棒杰股份 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2812, 000612, 焦作万方 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/d

---------- 正在处理股票：2885, 600199, 金种子酒 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2886, 600189, 泉阳泉 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2887, 600771, 广誉远 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2888, 002240, 盛新锂能 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2889, 600197, 伊力特 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2890, 603105, 芯能科技 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2891, 000546, 金圆股份 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2892, 002893, 华通热力 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2893, 600683, 京投发展 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
---------- 正在处理股票：2894, 002176, 江特电机 -----------
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/docu

KeyError: 'close'

In [38]:
error_list

[]

In [42]:
df_record = df_record.fillna(value=0)
df_record['平均涨幅'] = df_record['平均涨幅'].apply(lambda x:format(x, '.2%'))
df_record['平均跌幅'] = df_record['平均跌幅'].apply(lambda x:format(x, '.2%'))
df_record['涨率'] = df_record['涨率'].apply(lambda x:format(x, '.2%'))
df_record['跌率'] = df_record['跌率'].apply(lambda x:format(x, '.2%'))
df_record['期望'] = df_record['期望'].apply(lambda x:format(x, '.2%'))

df_record

,股票代码,股票名称,平均涨幅,平均跌幅,涨次,跌次,平次,共次,涨率,跌率,期望
1,600760,中航沈飞,3.60%,-2.96%,33,33,0,66,50.00%,50.00%,0.32%
1,000559,万向钱潮,3.86%,-3.35%,28,27,0,55,50.91%,49.09%,0.32%
1,002815,崇达技术,6.46%,-2.84%,11,11,0,22,50.00%,50.00%,1.81%
1,002484,江海股份,3.73%,-3.48%,14,25,0,39,35.90%,64.10%,-0.89%
1,000532,华金资本,4.07%,-4.49%,22,27,0,49,44.90%,55.10%,-0.65%
1,603009,北特科技,5.03%,-4.34%,37,30,0,67,55.22%,44.78%,0.84%
1,600405,动力源,3.56%,-5.74%,36,30,0,66,54.55%,45.45%,-0.67%
1,600284,浦东建设,5.14%,-4.36%,16,17,0,33,48.48%,51.52%,0.25%
1,002039,黔源电力,2.47%,-4.19%,5,7,0,12,41.67%,58.33%,-1.42%
1,003042,中农联合,7.30%,-9.19%,5,3,0,8,62.50%,37.50%,1.12%


In [43]:
import xlwt
df_record.to_excel('9292021.xls')

c:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  
